In [1]:
#I found the code in the Kaggle by Rutger Drubbel, which could be used to clean the dataset
from sklearn.preprocessing import Imputer
from sklearn.decomposition import PCA 
from sklearn.cluster import KMeans 
from sklearn.neighbors import KNeighborsClassifier
import nltk
from nltk.corpus import wordnet
PS = nltk.stem.PorterStemmer()
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import Imputer
from sklearn.decomposition import PCA # Principal Component Analysis module
from sklearn.cluster import KMeans # KMeans clustering 
from subprocess import check_output
import json
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
def load_tmdb_movies(path):
    df = pd.read_csv(path)
    df['release_date'] = pd.to_datetime(df['release_date']).apply(lambda x: x.date())
    json_columns = ['genres', 'keywords', 'production_countries', 'production_companies', 'spoken_languages']
    for column in json_columns:
        df[column] = df[column].apply(json.loads)
    return df

In [3]:
def pipe_flatten_names(keywords):
    return '|'.join([x['name'] for x in keywords])

def safe_access(container, index_values):
    # return a missing value rather than an error upon indexing/key failure
    result = container
    try:
        for idx in index_values:
            result = result[idx]
        return result
    except IndexError or KeyError:
        return pd.np.nan


def get_director(crew_data):
    directors = [x['name'] for x in crew_data if x['job'] == 'Director']
    return safe_access(directors, [0])


def pipe_flatten_names(keywords):
    return '|'.join([x['name'] for x in keywords])


def convert_to_original_format(movies):
    # Converts TMDb data to make it as compatible as possible with kernels built on the original version of the data.
    tmdb_movies = movies.copy()
    tmdb_movies.rename(columns=TMDB_TO_IMDB_SIMPLE_EQUIVALENCIES, inplace=True)
    tmdb_movies['title_year'] = pd.to_datetime(tmdb_movies['release_date']).apply(lambda x: x.year)
    # I'm assuming that the first production country is equivalent, but have not been able to validate this
    tmdb_movies['country'] = tmdb_movies['production_countries'].apply(lambda x: safe_access(x, [0, 'name']))
    tmdb_movies['language'] = tmdb_movies['spoken_languages'].apply(lambda x: safe_access(x, [0, 'name']))
    tmdb_movies['director_name'] = credits['crew'].apply(get_director)
    tmdb_movies['actor_1_name'] = credits['cast'].apply(lambda x: safe_access(x, [1, 'name']))
    tmdb_movies['actor_2_name'] = credits['cast'].apply(lambda x: safe_access(x, [2, 'name']))
    tmdb_movies['actor_3_name'] = credits['cast'].apply(lambda x: safe_access(x, [3, 'name']))
    tmdb_movies['genres'] = tmdb_movies['genres'].apply(pipe_flatten_names)
    tmdb_movies['plot_keywords'] = tmdb_movies['plot_keywords'].apply(pipe_flatten_names)
    return tmdb_movies

In [5]:
movies = load_tmdb_movies("C:/Users/guoshuhui/Desktop/542/homework/hw5/tmdb_5000_movies.csv")

In [6]:
df_new = pd.concat([movies], axis=1)
df_new['genres'] = df_new['genres'].apply(pipe_flatten_names)
df_new['keywords'] = df_new['keywords'].apply(pipe_flatten_names)
df_new['production_companies'] = df_new['production_companies'].apply(pipe_flatten_names)
df_new['production_countries'] = df_new['production_countries'].apply(pipe_flatten_names)
df_new['spoken_languages'] = df_new['spoken_languages'].apply(pipe_flatten_names)

In [7]:
df = df_new.copy()

In [8]:
df_use = df.reset_index(drop=True)
df_use.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,Action|Adventure|Fantasy|Science Fiction,http://www.avatarmovie.com/,19995,culture clash|future|space war|space colony|so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,Ingenious Film Partners|Twentieth Century Fox ...,United States of America|United Kingdom,2009-12-10,2787965087,162.0,English|Español,Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,Adventure|Fantasy|Action,http://disney.go.com/disneypictures/pirates/,285,ocean|drug abuse|exotic island|east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,Walt Disney Pictures|Jerry Bruckheimer Films|S...,United States of America,2007-05-19,961000000,169.0,English,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,Action|Adventure|Crime,http://www.sonypictures.com/movies/spectre/,206647,spy|based on novel|secret agent|sequel|mi6|bri...,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,Columbia Pictures|Danjaq|B24,United Kingdom|United States of America,2015-10-26,880674609,148.0,Français|English|Español|Italiano|Deutsch,Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,Action|Crime|Drama|Thriller,http://www.thedarkknightrises.com/,49026,dc comics|crime fighter|terrorist|secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,Legendary Pictures|Warner Bros.|DC Entertainme...,United States of America,2012-07-16,1084939099,165.0,English,Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,Action|Adventure|Science Fiction,http://movies.disney.com/john-carter,49529,based on novel|mars|medallion|space travel|pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,Walt Disney Pictures,United States of America,2012-03-07,284139100,132.0,English,Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [9]:
liste_genres = set()
for s in df['genres'].str.split('|'):
    liste_genres = set().union(s, liste_genres)
liste_genres = list(liste_genres)
liste_genres.remove('')

In [10]:
for genre in liste_genres:
    df_use[genre] = df['genres'].str.contains(genre).apply(lambda x:1 if x else 0)

In [11]:
liste_keywords = set()
for s in df['keywords'].str.split('|'):
    liste_keywords = set().union(s, liste_keywords)
liste_keywords = list(liste_keywords)
liste_keywords.remove('')

In [12]:
for keywords in liste_keywords:
    df_use[keywords] = df['keywords'].str.contains(keywords).apply(lambda x:1 if x else 0)

In [13]:
liste_production_companies = set()
for s in df['production_companies'].str.split('|'):
    liste_production_companies = set().union(s, liste_production_companies)
liste_production_companies = list(liste_production_companies)
liste_production_companies.remove('')
for production_companies in liste_production_companies:
    df_use[production_companies] = df['production_companies'].str.contains(production_companies).apply(lambda x:1 if x else 0)

df_use.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,Kinberg Genre,Axon Films,Eurimages,Merlin Films,Eigerwand Pictures,InVenture Entertainment,Gaylord Films,Corus,TMS Entertainment,Hollywood Partners
0,237000000,Action|Adventure|Fantasy|Science Fiction,0,19995,culture clash|future|space war|space colony|so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",0,Ingenious Film Partners|Twentieth Century Fox ...,...,0,0,0,0,0,0,0,0,0,0
1,300000000,Adventure|Fantasy|Action,0,285,ocean|drug abuse|exotic island|east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",0,Walt Disney Pictures|Jerry Bruckheimer Films|S...,...,0,0,0,0,0,0,0,0,0,0
2,245000000,Action|Adventure|Crime,0,206647,spy|based on novel|secret agent|sequel|mi6|bri...,en,Spectre,A cryptic message from Bond’s past sends him o...,0,Columbia Pictures|Danjaq|B24,...,0,0,0,0,0,0,0,0,0,0
3,250000000,Action|Crime|Drama|Thriller,0,49026,dc comics|crime fighter|terrorist|secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,0,Legendary Pictures|Warner Bros.|DC Entertainme...,...,0,0,0,0,0,0,0,0,0,0
4,260000000,Action|Adventure|Science Fiction,0,49529,based on novel|mars|medallion|space travel|pri...,en,John Carter,"John Carter is a war-weary, former military ca...",0,Walt Disney Pictures,...,0,0,0,0,0,0,0,0,0,0


In [15]:
liste_production_countries = set()
for s in df['production_countries'].str.split('|'):
    liste_production_countries = set().union(s, liste_production_countries)
liste_production_countries = list(liste_production_countries)
liste_production_countries.remove('')

for production_countries in liste_production_countries:
    df_use[production_countries] = df['production_countries'].str.contains(production_countries).apply(lambda x:1 if x else 0)

df_use.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,Mexico,Egypt,Fiji,Serbia,United Kingdom,Malaysia,Austria,China,Kyrgyz Republic,Japan
0,237000000,Action|Adventure|Fantasy|Science Fiction,0,19995,culture clash|future|space war|space colony|so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",0,Ingenious Film Partners|Twentieth Century Fox ...,...,0,0,0,0,1,0,0,0,0,0
1,300000000,Adventure|Fantasy|Action,0,285,ocean|drug abuse|exotic island|east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",0,Walt Disney Pictures|Jerry Bruckheimer Films|S...,...,0,0,0,0,0,0,0,0,0,0
2,245000000,Action|Adventure|Crime,0,206647,spy|based on novel|secret agent|sequel|mi6|bri...,en,Spectre,A cryptic message from Bond’s past sends him o...,0,Columbia Pictures|Danjaq|B24,...,0,0,0,0,1,0,0,0,0,0
3,250000000,Action|Crime|Drama|Thriller,0,49026,dc comics|crime fighter|terrorist|secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,0,Legendary Pictures|Warner Bros.|DC Entertainme...,...,0,0,0,0,0,0,0,0,0,0
4,260000000,Action|Adventure|Science Fiction,0,49529,based on novel|mars|medallion|space travel|pri...,en,John Carter,"John Carter is a war-weary, former military ca...",0,Walt Disney Pictures,...,0,0,0,0,0,0,0,0,0,0


In [16]:
liste_spoken_languages = set()
for s in df['spoken_languages'].str.split('|'):
    liste_spoken_languages = set().union(s, liste_spoken_languages)
liste_spoken_languages = list(liste_spoken_languages)
liste_spoken_languages.remove('')
liste_spoken_languages
df_use.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,Mexico,Egypt,Fiji,Serbia,United Kingdom,Malaysia,Austria,China,Kyrgyz Republic,Japan
0,237000000,Action|Adventure|Fantasy|Science Fiction,0,19995,culture clash|future|space war|space colony|so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",0,Ingenious Film Partners|Twentieth Century Fox ...,...,0,0,0,0,1,0,0,0,0,0
1,300000000,Adventure|Fantasy|Action,0,285,ocean|drug abuse|exotic island|east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",0,Walt Disney Pictures|Jerry Bruckheimer Films|S...,...,0,0,0,0,0,0,0,0,0,0
2,245000000,Action|Adventure|Crime,0,206647,spy|based on novel|secret agent|sequel|mi6|bri...,en,Spectre,A cryptic message from Bond’s past sends him o...,0,Columbia Pictures|Danjaq|B24,...,0,0,0,0,1,0,0,0,0,0
3,250000000,Action|Crime|Drama|Thriller,0,49026,dc comics|crime fighter|terrorist|secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,0,Legendary Pictures|Warner Bros.|DC Entertainme...,...,0,0,0,0,0,0,0,0,0,0
4,260000000,Action|Adventure|Science Fiction,0,49529,based on novel|mars|medallion|space travel|pri...,en,John Carter,"John Carter is a war-weary, former military ca...",0,Walt Disney Pictures,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df_use.to_csv("use_data.csv", encoding="UTF-8", index=False)